In [1]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [2]:
# データのパス
data_path = 'data/processed/android_cleaned_mecab.csv'

# データの読み込み
df = pd.read_csv(data_path)

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

データサイズ: (1000, 18)


,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review,positive_review_keywords,negative_review_keywords
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...,[],"['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し..."
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,-,- 指摘した問題が改善されていない,[],"['指摘', 'し', '問題', '改善', 'さ', 'れ', 'い']"
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,NaN,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,-,"['Wi-Fi', 'ある', '場所', '自動的', '接続', 'できる', '便利']",[]
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,NaN,9.6.2,0,[],-,CMが多く、5秒から30秒の動画が流れる,[],"['CM', '多く', '5秒', '30秒', '動画', '流れる']"
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-,"['ネット', '回線', '無く', 'GMOWiFi', '問題', '検索', 'ネッ...",[]


In [3]:
# データのnegative_review_keywordsに対してだけ，Topic Modelingを行う

negative_review_keywords = df['negative_review_keywords'].dropna().tolist()

In [4]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [5]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# umapとhdbscanのモデルを作成
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)

# BERTopicモデルを作成

topic_model = BERTopic(
    language="japanese",
    calculate_probabilities=False,
    verbose=True,
    nr_topics="20",
    umap_model=umap_model, 
    hdbscan_model=hdbscan_model
    )

topics, probs = topic_model.fit_transform(negative_review_keywords)

2024-12-19 11:27:54,622 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-12-19 11:28:01,442 - BERTopic - Embedding - Completed ✓
2024-12-19 11:28:01,443 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2024-12-19 11:28:06,080 - BERTopic - Dimensionality - Completed ✓
2024-12-19 11:28:06,080 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-19 11:28:06,103 - BERTopic - Cluster - Completed ✓
2024-12-19 11:28:06,104 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-19 11:28:06,121 - BERTopic - Representation - Completed ✓
2024-12-19 11:28:06,122 - BERTopic - Topic reduction - Reducing number of topics
2024-12-19 11:28:06,136 - BERTopic - Topic reduction - Reduced number of topics from 14 to 14


In [21]:
# モデルをsafetensorsで保存する
topic_model.save('models/android_topic_model', serialization="safetensors", save_embedding_model=True, save_ctfidf=True)

In [6]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,145,-1_いる_不満_こと_ポイント,"[いる, 不満, こと, ポイント, なっ, 感じ, ゲーム, よう, する, ため]","[['いら', 'ミッション', 'せい', '他', 'アプリ', '使え', 'なり',..."
1,0,369,0____,"[, , , , , , , , , ]","[[], [], []]"
2,1,100,1_広告_れる_表示_多く,"[広告, れる, 表示, 多く, する, 閉じる, 詐欺, こと, ボタン, ロード]","[['広告', '読み込み', '時間', 'かかり', '再生', 'さ', 'れ', '..."
3,2,86,2_動画_再生_視聴_ポイント,"[動画, 再生, 視聴, ポイント, でき, 時間, こと, ロード, ため, 問題]","[['動画', '再生', 'さ', 'れ'], ['動画', '再生', 'さ', 'れ'..."
4,3,58,3_fi_wi_wifi_接続,"[fi, wi, wifi, 接続, 繋がら, タウン, スポット, いる, でき, ある]","[['タウン', 'Wi-Fi', 'Wi-Fi', '使え', 'こと', 'ない'], ..."
5,4,45,4_ポイント_交換_不満_こと,"[ポイント, 交換, 不満, こと, でき, 増やし, 減る, なく, 貯め, 感じ]","[['貯め', 'ポイント', '更新', 'さ', 'れ', '連絡', 'なく', 'ポ..."
6,5,35,5_クイズ_難しい_難し_難点,"[クイズ, 難しい, 難し, 難点, フリーズ, 上手く, わから, 感じる, 使い, にくい]","[['クイズ', '難し', '過ぎ'], ['クイズ', '難し', 'すぎる', '感じ..."
7,6,33,6_アプリ_こと_表示_停止,"[アプリ, こと, 表示, 停止, 古い, 消え, でき, 問題, 起動, アップデート]","[['アプリ', 'アップデート', '必要', 'メッセージ', '表示', 'さ', '..."
8,7,33,7_接続_自動_繋がり_悪い,"[接続, 自動, 繋がり, 悪い, vpn, つながり, でき, スタンプ, wifi, 問題]","[['接続', '自動', 'でき', 'こと', 'ある'], ['自動', '接続', ..."
9,8,22,8_アップデート_インストール_バージョンアップ_なっ,"[アップデート, インストール, バージョンアップ, なっ, エラー, ソフト, でき, た...","[['2024年', '5月15日', 'アップデート', '更新', 'さ', 'れ', ..."


In [20]:
# トピックIDと名前の対応付け
topic_mapping = topic_model.get_topic_info().set_index('Topic')['Name'].to_dict()

# topics と probs はそれぞれリストまたは NumPy 配列と仮定
# negative_review_keywords の行数と topics, probs の長さが一致している前提
df['topic'] = topics  # トピックIDを新しい列 'topic' として追加
df['topic_probability'] = probs  # トピック確率を新しい列 'topic_probability' として追加
df['topic_name'] = df['topic'].map(topic_mapping)  # トピックIDをトピック名に変換

# dfを保存
df.to_csv('data/processed/android_cleaned_mecab_with_topics.csv', index=False)

In [7]:
topic_model.visualize_topics()

In [8]:
topic_model.visualize_hierarchy()

In [9]:
topic_model.visualize_barchart(
    top_n_topics=14,
    width=500,  # 幅を広げる
)

In [12]:
# topics, probs = topic_model.fit_transform(negative_review_keywords)の結果を元のデータに新たなカラムとして追加

df['topic'] = topics

df

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review,positive_review_keywords,negative_review_keywords,topic
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...,[],"['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し...",6
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,-,- 指摘した問題が改善されていない,[],"['指摘', 'し', '問題', '改善', 'さ', 'れ', 'い']",-1
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,NaN,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,-,"['Wi-Fi', 'ある', '場所', '自動的', '接続', 'できる', '便利']",[],0
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,NaN,9.6.2,0,[],-,CMが多く、5秒から30秒の動画が流れる,[],"['CM', '多く', '5秒', '30秒', '動画', '流れる']",2
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-,"['ネット', '回線', '無く', 'GMOWiFi', '問題', '検索', 'ネッ...",[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,a9462ebd-6ac7-4920-8285-5922664d0592,印南勤,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-04-10T15:56:08.523Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,つながりがかなり良い,2024-04-14T06:40:17.733Z,いつもアプリをお使いいただき、ありがとうございます！\n今後とも、タウンWiFiをよろしくお...,8.31.1,0,"[{'criteria': 'vaf_phase1_file_sharing', 'rati...",つながりがかなり良い,-,"['つながり', '良い']",[],0
996,0733b493-a295-4cee-931c-144fcc1e44a0,三河のどか,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-04-10T08:51:49.680Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,ビックリするほどバッテリーの消耗が激しい。使わない時もバックグラウンドで動いているようで使わ...,NaN,NaN,NaN,8,[],-,- バッテリーの消耗が激しい,[],"['バッテリー', '消耗', '激しい']",-1
997,90f66974-f8f9-44ef-9079-707cfd207c82,村上稔,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-04-10T08:24:58.380Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,又近い内に,NaN,NaN,NaN,0,[],-,-,[],[],0
998,c417d407-b5f8-40c0-b074-f331e8aab9a6,沼津のトシ,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-04-09T23:26:14.886Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,スタンプをせっせと貯めていますが、いかんせん交換率が凄く低くて、3万ポイントがアマゾンポイン...,NaN,NaN,8.31.1,3,[],-,スタンプの交換率が低く、3万ポイントをアマゾンポイントに変えると300ポイントで、13万ポイ...,[],"['スタンプ', '交換', '率', '低く', '3', '万', 'ポイント', 'ア...",4
